# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
# Output File (CSV)
output_data_file = "../WeatherPy/output_data/cities.csv" 

# Read the csv file 
weather_data_df = pd.read_csv(output_data_file)

# View weather data in dataframe
weather_data_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Faanui,-16.48,-151.75,78.57,76,13,10.11,PF,1603378917
1,Thompson,55.74,-97.86,21.20,92,75,4.70,CA,1603378919
2,Cape Town,-33.93,18.42,70.00,64,0,20.80,ZA,1603378884
3,Goderich,43.75,-81.72,48.00,88,100,7.63,CA,1603378880
4,Kargil,34.57,76.10,41.16,56,3,6.26,IN,1603378924


### Humidity Heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
locations_complete = locations.dropna()
humidity = weather_data_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=3)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_complete, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
# Narrow down the cities to fit weather conditions.
vacation_df = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) 
                                  & (weather_data_df['Max Temp'] < 80) 
                                  & (weather_data_df['Wind Speed'] < 10)
                                  & (weather_data_df['Cloudiness'] == 0)
                                  & (weather_data_df['Humidity'] < 70)
                                 ]

# Drop any rows will null values.
vacation_df_complete = vacation_df.dropna()

### Hotel Map


In [6]:
# Store into variable named hotel_df
hotel_df = vacation_df_complete.copy()

#Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''

hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,Plettenberg Bay,-34.05,23.37,71.28,60,0,7.81,ZA,1603378918,
46,Hermanus,-34.42,19.23,73.99,64,0,5.99,ZA,1603378984,
91,Mizdah,31.45,12.98,76.51,31,0,8.52,LY,1603379046,
97,Castro,-24.79,-50.01,74.91,50,0,9.62,BR,1603379053,
234,Patti,31.28,74.86,78.80,60,0,3.27,IN,1603379238,
275,Abu Samrah,35.30,37.18,71.76,28,0,1.88,SY,1603379296,
354,Skotina,40.01,22.55,75.20,36,0,1.54,GR,1603379408,
407,Jacobabad,28.28,68.44,75.47,16,0,3.06,PK,1603379485,
472,Waddān,29.16,16.14,79.05,25,0,2.04,LY,1603379573,
497,Moindou,-21.69,165.68,77.00,65,0,3.36,NC,1603379607,


In [10]:
# Set the base url to serch
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    city = row['City']

    # get latitude and longitude from df
    hotel_loc = row[['Latitude', 'Longitude']]

    # add keyword to params dict
    params['location'] = f'{hotel_loc[0]}, {hotel_loc[1]}' 

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
                
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 15: Plettenberg Bay.
Closest hotel is Thanda Vista B&B.
------------
Retrieving Results for Index 46: Hermanus.
Closest hotel is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 91: Mizdah.
Closest hotel is فندق بنيه تحتيه.
------------
Retrieving Results for Index 97: Castro.
Closest hotel is CHACARA BAILLY.
------------
Retrieving Results for Index 234: Patti.
Closest hotel is Swami dera Geeta mandir.
------------
Retrieving Results for Index 354: Skotina.
Closest hotel is Pliades Hotel.
------------
Retrieving Results for Index 407: Jacobabad.
Closest hotel is Sardar Hotel.
------------
Retrieving Results for Index 472: Waddān.
Closest hotel is محل الشريف محمد العايب.
------------
Retrieving Results for Index 497: Moindou.
Closest hotel is Gite de la RYAWA.
------------
Retrieving Results for Index 542: Vallenar.
Closest hotel is Humacao Bed & Breakfast.
------------


In [11]:
# Drop missing hotel information
miss_data = hotel_df.loc[hotel_df['Hotel Name'] == '']
index = miss_data.index
hotel_df = hotel_df.drop(index)

In [12]:
# Hotel info
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,Plettenberg Bay,-34.05,23.37,71.28,60,0,7.81,ZA,1603378918,Thanda Vista B&B
46,Hermanus,-34.42,19.23,73.99,64,0,5.99,ZA,1603378984,Misty Waves Boutique Hotel
91,Mizdah,31.45,12.98,76.51,31,0,8.52,LY,1603379046,فندق بنيه تحتيه
97,Castro,-24.79,-50.01,74.91,50,0,9.62,BR,1603379053,CHACARA BAILLY
234,Patti,31.28,74.86,78.80,60,0,3.27,IN,1603379238,Swami dera Geeta mandir
354,Skotina,40.01,22.55,75.20,36,0,1.54,GR,1603379408,Pliades Hotel
407,Jacobabad,28.28,68.44,75.47,16,0,3.06,PK,1603379485,Sardar Hotel
472,Waddān,29.16,16.14,79.05,25,0,2.04,LY,1603379573,محل الشريف محمد العايب
497,Moindou,-21.69,165.68,77.00,65,0,3.36,NC,1603379607,Gite de la RYAWA
542,Vallenar,-28.57,-70.76,77.59,31,0,6.58,CL,1603379666,Humacao Bed & Breakfast


In [13]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations,
    info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))